In [1]:
import torch #Menyediakan fungsi dasar untuk operasi tensor dan komputasi berbasis GPU.
import torch.nn as nn #Menyediakan modul dan kelas untuk membangun jaringan saraf.
import torch.optim as optim #Menyediakan algoritma pengoptimalan untuk melatih model.
from torch.utils.data import DataLoader, random_split #Menyediakan kelas dan fungsi untuk memuat dan memproses data.
import torchvision #Menyediakan kumpulan data, model, dan transformasi gambar yang telah dibuat sebelumnya.
import torchvision.transforms as transforms #Menyediakan transformasi gambar umum untuk augmentasi data.
from torch.optim.lr_scheduler import ReduceLROnPlateau #Menyediakan scheduler laju pembelajaran untuk menyesuaikan laju pembelajaran selama pelatihan.

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

#Transformasi Dataset

In [3]:
batch_size = 32
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
#Load Fashion MNIST Dataset

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.3MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 208kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.88MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 4.04MB/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [4]:
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#Split train dataset menjadi train dan validation

In [5]:
class CNN(nn.Module):
    def __init__(self, kernel_size=3, pooling_type='max'):
        super(CNN, self).__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_size, padding=padding)  # Input 1 channel (grayscale)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=padding)

        if pooling_type == 'max':
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        elif pooling_type == 'avg':
            self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Size of feature map after pooling: 7x7
        self.fc2 = nn.Linear(128, 10)  # 10 classes in Fashion MNIST
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
#Definisi Arsitektur CNN

In [ ]:
def train_and_evaluate(kernel_size, pooling_type, optimizer_type, epochs, early_stopping_patience=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CNN(kernel_size=kernel_size, pooling_type=pooling_type).to(device)
    criterion = nn.CrossEntropyLoss()

    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    elif optimizer_type == 'RMSProp':
        optimizer = optim.RMSprop(model.parameters(), lr=0.01)
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=0.001)
    #Optimizer

    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=False)

    best_loss = float('inf')
    patience_counter = 0

    print(f"Evaluating with params: {{'epochs': {epochs}, 'kernel_size': {kernel_size}, 'optimizer_type': '{optimizer_type}', 'pooling_type': '{pooling_type}'}}")

    for epoch in range(epochs):
        # Training Step
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
    #Learning Rate Scheduler

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        scheduler.step(val_loss)
        #Validation Step

        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print("Early stopping triggered.")
                break
        #Early Stopping

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy
    #Evaluasi Akurasi pada Test Dataset

#Training dan Evaluasi

kernel_sizes = [3, 5, 7]
pooling_types = ['max', 'avg']
optimizers = ['SGD', 'RMSProp', 'Adam']
epoch_list = [5, 50, 100, 250, 350]

results = []

for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        for optimizer in optimizers:
            for epochs in epoch_list:
                print(f"Running experiment with kernel_size={kernel_size}, pooling_type={pooling_type}, optimizer={optimizer}, epochs={epochs}")
                accuracy = train_and_evaluate(kernel_size, pooling_type, optimizer, epochs, early_stopping_patience=10)
                results.append((kernel_size, pooling_type, optimizer, epochs, accuracy))

# Menampilkan Hasil Akhir
print("\nHasil Akhir Eksperimen:")
for result in results:
    print(f"Kernel Size: {result[0]}, Pooling: {result[1]}, Optimizer: {result[2]}, Epochs: {result[3]}, Accuracy: {result[4]:.2f}%")
# Menampilkan Hasil Akhir

#Eksperimen dengan Semua Kombinasi Parameter

Running experiment with kernel_size=3, pooling_type=max, optimizer=SGD, epochs=5
Evaluating with params: {'epochs': 5, 'kernel_size': 3, 'optimizer_type': 'SGD', 'pooling_type': 'max'}


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/5, Loss: 0.6210, Val Loss: 0.3640
Epoch 2/5, Loss: 0.3856, Val Loss: 0.3066
Epoch 3/5, Loss: 0.3249, Val Loss: 0.2868
Epoch 4/5, Loss: 0.2872, Val Loss: 0.2631
Epoch 5/5, Loss: 0.2606, Val Loss: 0.2471
Accuracy: 90.77%
Running experiment with kernel_size=3, pooling_type=max, optimizer=SGD, epochs=50
Evaluating with params: {'epochs': 50, 'kernel_size': 3, 'optimizer_type': 'SGD', 'pooling_type': 'max'}
Epoch 1/50, Loss: 0.6148, Val Loss: 0.3564
Epoch 2/50, Loss: 0.3821, Val Loss: 0.3197
Epoch 3/50, Loss: 0.3165, Val Loss: 0.2695
Epoch 4/50, Loss: 0.2873, Val Loss: 0.2524
Epoch 5/50, Loss: 0.2548, Val Loss: 0.2481
Epoch 6/50, Loss: 0.2371, Val Loss: 0.2381
Epoch 7/50, Loss: 0.2217, Val Loss: 0.2336
Epoch 8/50, Loss: 0.2083, Val Loss: 0.2368
Epoch 9/50, Loss: 0.1973, Val Loss: 0.2358
Epoch 10/50, Loss: 0.1844, Val Loss: 0.2391
Epoch 11/50, Loss: 0.1739, Val Loss: 0.2404
Epoch 12/50, Loss: 0.1690, Val Loss: 0.2440
Epoch 13/50, Loss: 0.1615, Val Loss: 0.2381
Epoch 14/50, Loss: 0.11